# Bayesian Inference
Bayes' theorem provides a general recipe to estimate the value of the parameter
$\boldsymbol{\theta}$ given observed data
$\boldsymbol{Y}$:

```{math}
:label: eq:posterior_dist
\underbrace{p(\boldsymbol{\theta} \mid \boldsymbol{Y})}_{\text{posterior}} = \frac{\overbrace{p(\boldsymbol{Y} \mid \boldsymbol{\theta})}^{\text{likelihood}}\; \overbrace{p(\boldsymbol{\theta})}^{\text{prior}}}{\underbrace{{p(\boldsymbol{Y})}}_{\text{marginal likelihood}}}

```

$ \underbrace{p(\boldsymbol{\theta} \mid \boldsymbol{Y})}_{\text{posterior}} = \frac{\overbrace{p(\boldsymbol{Y} \mid \boldsymbol{\theta})}^{\text{likelihood}}\; \overbrace{p(\boldsymbol{\theta})}^{\text{prior}}}{\underbrace{{p(\boldsymbol{Y})}}_{\text{marginal likelihood}}} $

* The likelihood function links the observed data with the unknown parameters 
* The prior distribution represents the uncertainty about the parameters before observing the data $\boldsymbol{Y}$. 
* The posterior distribution, is the joint distribution over all the parameters in the model (conditioned on the
observed data).

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
# Import pyMC3 and also arviz for visualisation
import pymc as pm
import arviz as az
import sympy as sp
import pandas as pd
import numpy as np
import statsmodels.api as sm


import symseaman as ss
from symseaman.substitute_dynamic_symbols import lambdify, remove_bis_eq, run, standard_substitutes
from symseaman.seaman_symbols import *
from vct.bis_system import BisSystem

In [ ]:
import sympy as sp
from symseaman.seaman_symbols import *
from symseaman.substitute_dynamic_symbols import run, lambdify, standard_substitutes, remove_bis, remove_bis_eq, standard_substitutes_eq, lower_and_abs
from symseaman.seaman_symbol import BisSymbol
import symseaman as ss
from symseaman.seaman_symbol import expand_bis
from symseaman.shipdict import ShipDict
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', -1)
import numpy as np
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = 9,7
#plt.style.use('bmh')
import vct.bokeh_plotting as bokeh_plotting
from numpy.testing import assert_almost_equal
from scipy.interpolate import interp1d
from symseaman.calculate import calculate_ship
from symseaman.to_matrix import matrix_form, matrix_form_standard, exclude, exclude_derivatives
from functools import reduce
from operator import add
from vct.regression_ols import Regression, RegressionPipeline
import dill
import seaborn as sns
from myst_nb import glue
import matplotlib.pyplot as plt
plt.style.use('jupyter')

In [ ]:
# Read configs:
import anyconfig
globals_path = "../../conf/base/globals.yml"
global_variables = anyconfig.load(globals_path)
ships = global_variables["ships"]
from vct.bis_system import BisSystem
import statsmodels.api as sm
from scipy.stats import norm
from vct.regression_ols import data_to_bis
from scipy.stats import norm, multivariate_normal
import regression_ols, regression_ols_split

In [ ]:
%reload_kedro
ship_name=ships[0]
shipdict = catalog.load(f"{ship_name}.shipdict_base")
ship_dict_from_regression = catalog.load(f"{ ship_name }.ship_dict_from_regression")

df_VCT = catalog.load(f"{ship_name}.df_VCT")
df_VCT['g'] = df_VCT['g'].fillna(method='pad')
df_VCT['rho'] = df_VCT['rho'].fillna(method='pad')
df_VCT['beta_deg'] = np.rad2deg(df_VCT['beta'])

df_VCT['fx_hull'] = df_VCT['fx']  # Not true but works for this example...
df_VCT['fy_hull'] = df_VCT['fy']  # Not true but works for this example...
df_VCT['mz_hull'] = df_VCT['mz']  # Not true but works for this example...

thrust_keys = catalog.load(f"params:{ship_name}.thrust_keys")

if len(thrust_keys) > 0:
    df_VCT['tprop'] = df_VCT[thrust_keys[0]]
else:
    df_VCT['tprop'] = 0
    thrust_keys=['tprop']
    
derivatives = catalog.load(f"{ship_name}.derivatives")["OLS"]
derivatives_std = catalog.load(f"{ship_name}.derivatives_std")["OLS"]

In [ ]:
df_ = df_VCT.groupby(by='test type').get_group('Drift angle')
pipeline, df_bis = regression_ols.fit(df_VCT=df_VCT, shipdict=shipdict)
pre_set_derivatives_0 = {}
regression = pipeline['drift']
y,X=regression.label_and_features(derivatives=pre_set_derivatives_0, meta_data=pipeline.meta_data)

In [ ]:
pipeline_split, _ = regression_ols_split.fit(df_VCT=df_VCT, shipdict=shipdict)

## Informative priors

It was shown that the OLS regression can produce a model that fits the data well, but that the individual hydrodynamic derivatives within the model are very uncertain due to muticollinearity.
In the OLS regression, very little assumptions regarding the actual values of the hydrodynamic derivatives were made prior to the regression. This is a very usefull property when no prior understanding of the data and what the parameters in the model represent is available. 

In this case however, we can apply some reasoning with our prior knowledge about the data, which can be used to build informative priors to be later used in Bayesian inference.
The model ({numref}`eqsway`) has a connection to the underlying hydrodynamics, where:
* $Y_{uv}$ represents the hydrodynamic lift
* $Y_{uuv}$ represents the speed dependent part of the hydrodynamic lift
* $Y_{v|v|}$ represents the cross flow drag

This knowledge can be used to make an adopted regression. Often a strategy can be adopted where the regression is split into two sub regressions - one for each speed. The estimated hydrodynamic lift coefficients for the two sub regressions are shown in {numref}`regressionsplit`. It seems that the absolute value of hydrodynamic derivative is a bit smaller for the higher speed than for the lower speed. $Y_{uv}$ and $Y_{uuv}$ can be estimated by fitting a line through these two points.

```{glue:figure} regressionsplit
:name: "regressionsplit"

Regression split.
```

In [ ]:
df_yuv = pd.DataFrame(pipeline_split.get_suffixed_derivatives(derivative='yuv'))
df_yuv['yvav'] = pipeline_split.get_suffixed_derivatives(derivative='yvav')
Vs_ = df_yuv.index
df_yuv['yuv_model'] = pipeline_split.derivatives['yuv'] + pipeline_split.derivatives['yuuv']*Vs_
df_yuv['yvav_model'] = pipeline_split.derivatives['yvav']

fig,ax=plt.subplots()
df_yuv.plot(y='yuv', label='itterations', style='x', zorder=10, ax=ax)
df_yuv.plot(y='yuv_model', label='model', style='--', ax=ax)
ax.set_ylabel(r"$Y_{uv}'' \left( V'' \right) $")
ax.set_xlabel(r"$ V'' [-] $")
ax.set_ylim((-1.13,0))
glue("regressionsplit",fig, display=False)


{numref}`informativepriors` shows the probability density functions (PDF) for hydrodynamic derivatives estimated with OLS regression on the whole dataset and the splitted regression.  

```{glue:figure} informativepriors
:name: "informativepriors"

PDF:s for the estimated derivatives with OLS regression and splitted OLS regression.
```

In [ ]:
columns = list(X.columns)

#sigmas_ = [derivatives_std[key] for key in columns]
#mus_ = [derivatives[key] for key in columns]

sigmas_ = [pipeline_split.stds[key] for key in columns]
mus_ = [pipeline_split.derivatives[key] for key in columns]
data = df_
sigma_ = 0.0001*data['fy'].abs().max()

In [ ]:
columns_sympy = {
    'yuv':Y_uv,
    'yuuv':Y_uuv,
    'yvav':Y_vav,
}
columns_latex = {key:f"${sp.latex(value)}$" for key,value in columns_sympy.items()}

#rvs_split_OLS = {key:norm(loc=derivatives[key], scale=derivatives_std[key]) for key in columns}
rvs_split_OLS = {key:norm(loc=pipeline_split.derivatives[key], scale=pipeline_split.stds[key]) for key in columns}
rvs_OLS = {key:norm(loc=regression.regression.params[key], scale=regression.regression.bse[key]) for key in columns}


fig,axes=plt.subplots(ncols=len(rvs_OLS))
for (key, rv),ax in zip(rvs_split_OLS.items(), axes.flatten()):
    
    rv_OLS = rvs_OLS[key]
    x_ = np.linspace(rv_OLS.mean() - 3*rv_OLS.std(), rv_OLS.mean() + 3*rv_OLS.std(), 200)
   
    ax.plot(x_, rv_OLS.pdf(x_), label='OLS')
    ax.plot(x_, rv.pdf(x_), label='OLS splitted')
    
        
    ax.set_xlabel(columns_latex[key])
    
axes[0].legend()
glue("informativepriors", fig, display=False)


The PDF:s from the splitted OLS regression can now be used as informative priors to do Bayesian inference of the parameters.

In [ ]:
basic_model = pm.Model()



with basic_model:
    # Priors for unknown model parameters
    parameters = pm.Normal("parameters", mu=mus_, 
                           sigma=sigmas_, 
                           shape=len(columns))
    
    sigma = pm.HalfNormal("sigma", sigma=sigma_)

    # Expected value of outcome
    mu = pm.math.dot(X, parameters)

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("fy_obs", mu=mu, sigma=sigma, observed=y)
    #ν = pm.Uniform("ν", 1, 10)
    #Y_obs = pm.StudentT("fy_obs", nu=ν, mu=mu, sigma=sigma, observed=y)

In [ ]:
with basic_model:
    prior = pm.sample_prior_predictive(samples=5000)
    trace = pm.sample(draws=10000)

In [ ]:
with basic_model:
    az.plot_posterior(trace,
                  var_names=['parameters','sigma'],
                  textsize=10,
                  point_estimate='mean',
                  rope_color='black',
                  figsize=(14,4))

In [ ]:
az.summary(trace)

In [ ]:
df_parameters_posterior = pd.DataFrame(data = np.concatenate((trace.posterior['parameters'].data[0],trace.posterior['parameters'].data[1])), columns=columns)

```{glue:figure} bayesianinference
:name: "bayesianinference"

PDF:s for the estimated derivatives with OLS regression, splitted OLS regression and Bayesian inference.
```

In [ ]:
fig,axes=plt.subplots(ncols=len(rvs_OLS))
for (key, rv),ax in zip(rvs_split_OLS.items(), axes.flatten()):
    
    rv_OLS = rvs_OLS[key]
    x_ = np.linspace(rv_OLS.mean() - 3*rv_OLS.std(), rv_OLS.mean() + 3*rv_OLS.std(), 200)
   
    ax.plot(x_, rv_OLS.pdf(x_), label='OLS')
    ax.plot(x_, rv.pdf(x_), label='OLS splitted')
    sns.kdeplot(df_parameters_posterior[key], ax=ax, label='Bayesian inference')
        
    ax.set_xlabel(columns_latex[key])
    
axes[1].legend(loc='center')
glue("bayesianinference", fig, display=False)

In [ ]:
means = (trace.posterior['parameters'].data[0].mean(axis=0) + trace.posterior['parameters'].data[1].mean(axis=0))/2
means = {key:mean for key,mean in zip(columns,means)}

In [ ]:
stds = (trace.posterior['parameters'].data[0].std(axis=0) + trace.posterior['parameters'].data[1].std(axis=0))/2
stds = {key:std for key,std in zip(columns,stds)}

In [ ]:
df_parameters = pd.concat([
pd.Series(means, name='Bayesian'),
pd.Series({key:derivatives[key] for key in means.keys()}, name='splitted OLS'),
pd.Series({key:regression.regression.params[key] for key in means.keys()}, name='OLS'),
    
], axis=1).transpose()

```{glue:figure} estimations
:name: "estimations"

Estimated hydrodynamic derivatives with the various methods.
```

In [ ]:
fig,ax=plt.subplots()
df_parameters.rename(columns=columns_latex).plot.barh(ax=ax)
glue("estimations",fig, display=False)